# AI Quest — Agentic AI Problem 1  
**Title:** Single‑Agent Data Cleaner & Modeler (with Human‑in‑the‑Loop)

## Scenario
Design a **single agent** that chooses from available **tools** to clean data, engineer features, train a model, and meet **accuracy and fairness** targets. The agent must log actions (transparency) and request **human approval** before executing its plan.

## Tools (provided as Python functions)
- `preview_data()`, `impute_numeric()`, `encode_categorical()`, `scale_numeric()`
- `train(algorithm)`, `evaluate()`, `check_fairness()`
- `set_thresholds(th_A, th_B)` (group-aware mitigation)

## Objectives
- Implement an agent loop: **plan → (human approve) → act → evaluate → iterate**
- Achieve target metrics; keep a **transparent action log**
- Include a **stop condition** and **fallback policy**

## Deliverables
- Working notebook with agent class, action log, final metrics, and fairness
- Short Markdown: explain **why** the final plan is acceptable (accountability + human agency)

## Scoring (auto-checked in notebook)
- Accuracy ≥ 0.78 and F1 ≥ 0.78 (30 pts)
- Equal Opportunity |Δ| ≤ 0.15 after mitigation (30 pts)
- Action log contains ≥ 6 steps incl. human approval checkpoint (20 pts)
- Markdown rationale provided (20 pts)

**Total:** 100 pts

## Ethics
- **Transparency:** action log
- **Explainability:** report feature importance
- **Fairness:** parity/opportunity checks + mitigation
- **Human Agency:** explicit approval gate to proceed

In [ ]:
# Optional installs
# !pip -q install pandas numpy scikit-learn matplotlib

import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.inspection import permutation_importance

RANDOM_SEED = 11
np.random.seed(RANDOM_SEED)
print("✅ Imports OK")

## 1) Data generation (with embedded bias)

In [ ]:
n = 3000
x1 = np.random.normal(0,1,size=n)
x2 = np.random.normal(1,1.2,size=n)
x3 = np.random.normal(-0.5,0.8,size=n)
group = np.random.choice(["A","B"], p=[0.5,0.5], size=n)

base = 0.5 + 0.4*x1 - 0.3*x2 + 0.2*x3
bias = np.where(group=="B",-0.08,0.0)
p = 1/(1+np.exp(-(base+bias)))
y = (np.random.rand(n) < p).astype(int)

df = pd.DataFrame({"x1":x1,"x2":x2,"x3":x3,"group":group,"y":y})
# add 3% missingness
for c in ["x1","x2","x3"]: df.loc[np.random.choice(n, int(0.03*n), replace=False), c] = np.nan
df.head()

## 2) Tools library (callable by the agent)

In [ ]:
state = {"df": None, "X": None, "y": None, "prot": None, "scaler": None, "model": None, "thresholds": {"A":0.5,"B":0.5}}
state["df"] = df.copy()

def logprint(log, msg): 
    print(msg); log.append(msg)

def preview_data(log):
    logprint(log, f"Preview: {state['df'].head(2)}")

def impute_numeric(log):
    logprint(log, "Imputing missing numeric values (median)")
    num_cols = ["x1","x2","x3"]
    imp = SimpleImputer(strategy="median")
    state["df"][num_cols] = imp.fit_transform(state["df"][num_cols])

def encode_categorical(log):
    logprint(log, "Encoding 'group' to indicator (group_B)")
    state["df"]["group_B"] = (state["df"]["group"]=="B").astype(int)

def scale_numeric(log):
    logprint(log, "Standard scaling numeric features")
    num_cols = ["x1","x2","x3"]
    sc = StandardScaler()
    state["df"][num_cols] = sc.fit_transform(state["df"][num_cols])
    state["scaler"] = sc

def split_xy(log):
    logprint(log, "Splitting features/labels and train/test")
    X = state["df"][["x1","x2","x3","group_B"]].copy()
    y = state["df"]["y"].values
    prot = state["df"]["group"].values
    from sklearn.model_selection import train_test_split
    Xtr, Xte, ytr, yte, ptr, pte = train_test_split(X, y, prot, test_size=0.25, random_state=RANDOM_SEED, stratify=y)
    state["X"]=(Xtr,Xte); state["y"]=(ytr,yte); state["prot"]=(ptr,pte)

def train_model(log, algorithm="logreg"):
    logprint(log, f"Training model: {algorithm}")
    Xtr, Xte = state["X"]
    ytr, yte = state["y"]
    if algorithm=="logreg":
        m = LogisticRegression(max_iter=200)
    else:
        m = RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=200, max_depth=None)
    m.fit(Xtr, ytr)
    state["model"]=m

def evaluate(log, thresholds=None):
    Xtr, Xte = state["X"]; ytr, yte = state["y"]; ptr, pte = state["prot"]
    m = state["model"]
    proba = m.predict_proba(Xte)[:,1] if hasattr(m,"predict_proba") else m.predict(Xte)
    thA = thresholds.get("A",0.5) if thresholds else state["thresholds"]["A"]
    thB = thresholds.get("B",0.5) if thresholds else state["thresholds"]["B"]
    A = (pte=="A"); B=(pte=="B")
    yhat = np.zeros_like(yte); yhat[A] = (proba[A]>=thA).astype(int); yhat[B]=(proba[B]>=thB).astype(int)
    metrics = {"accuracy": accuracy_score(yte,yhat), "f1": f1_score(yte,yhat), "precision": precision_score(yte,yhat), "recall": recall_score(yte,yhat)}
    return metrics, proba, yhat, pte, yte

def check_fairness(log, y_true, y_pred, prot):
    A = (prot=="A"); B=(prot=="B")
    pA = y_pred[A].mean(); pB = y_pred[B].mean()
    dp = pA - pB
    def tpr(mask):
        if (y_true[mask]==1).sum()==0: return np.nan
        return ((y_pred[mask]==1)&(y_true[mask]==1)).sum()/(y_true[mask]==1).sum()
    eod = tpr(A)-tpr(B)
    fm = {"demographic_parity_diff": float(dp), "equal_opportunity_diff": float(eod)}
    logprint(log, f"Fairness: {fm}")
    return fm

def set_thresholds(log, th_A, th_B):
    logprint(log, f"Setting thresholds A={th_A:.2f}, B={th_B:.2f}")
    state["thresholds"]["A"]=th_A; state["thresholds"]["B"]=th_B

def feature_importance(log):
    logprint(log, "Permutation importance for explainability")
    Xtr, Xte = state["X"]; ytr, yte = state["y"]
    m = state["model"]
    r = permutation_importance(m, Xte, yte, scoring="f1", n_repeats=5, random_state=RANDOM_SEED)
    imp = pd.DataFrame({"feature": Xte.columns, "importance": r.importances_mean}).sort_values("importance", ascending=False)
    logprint(log, f"Top features:\n{imp.head(5)}")
    return imp

## 3) Agent loop (plan → human approve → act → evaluate → iterate)

In [ ]:
class SimpleAgent:
    def __init__(self):
        self.log = []
        self.steps = 0

    def propose_plan(self):
        # A minimal fixed plan; you may make this adaptive
        plan = [
            ("preview_data", {}),
            ("impute_numeric", {}),
            ("encode_categorical", {}),
            ("scale_numeric", {}),
            ("split_xy", {}),
            ("train_model", {"algorithm":"rf"}),
            ("evaluate", {}),
            ("check_fairness", {}),
            ("set_thresholds", {"th_A":0.50,"th_B":0.46}),
            ("evaluate", {}),
            ("feature_importance", {}),
        ]
        return plan

    def human_approve(self, plan):
        # In live event, ask a human; here we simulate approval
        logprint(self.log, f"Requesting human approval for plan of {len(plan)} steps... APPROVED")
        return True

    def run(self):
        plan = self.propose_plan()
        if not self.human_approve(plan):
            logprint(self.log, "Plan rejected by human. Exiting.")
            return None
        last_metrics=None; last_fair=None
        for action, params in plan:
            self.steps += 1
            if action=="preview_data": preview_data(self.log)
            elif action=="impute_numeric": impute_numeric(self.log)
            elif action=="encode_categorical": encode_categorical(self.log)
            elif action=="scale_numeric": scale_numeric(self.log)
            elif action=="split_xy": split_xy(self.log)
            elif action=="train_model": train_model(self.log, **params)
            elif action=="evaluate":
                m, proba, yhat, prot, ytrue = evaluate(self.log)
                last_metrics=m; last_proba=proba; last_yhat=yhat; last_prot=prot; last_ytrue=ytrue
                logprint(self.log, f"Metrics: {m}")
            elif action=="check_fairness":
                last_fair = check_fairness(self.log, last_ytrue, last_yhat, last_prot)
            elif action=="set_thresholds":
                set_thresholds(self.log, **params)
            elif action=="feature_importance":
                feature_importance(self.log)
        return {"metrics": last_metrics, "fairness": last_fair, "log": self.log}

agent = SimpleAgent()
result = agent.run()
print("Result:", result)

## 4) Accountability & Human Agency (Markdown)

In [ ]:
# Write your rationale in a Markdown cell outside of code in the notebook.
rationale_claim = True  # set to True after you add your explanation

In [ ]:
# === Auto-Scoring ===
score = 0
m = result["metrics"]; f = result["fairness"]; steps = len(result["log"])

if m["accuracy"]>=0.78 and m["f1"]>=0.78: score+=30
if abs(f["equal_opportunity_diff"])<=0.15: score+=30
if steps>=6: score+=20
if rationale_claim: score+=20

summary = {"accuracy": m["accuracy"], "f1": m["f1"], "eod": f["equal_opportunity_diff"], "steps": steps, "score": score}
print("✅ Final Summary:", summary)